In [27]:
import numpy as np
import pandas as pd
import math

df = pd.read_csv('toys.csv')
X = df['X']
Y = df['Y']

In [40]:
def sigmoid(w,b,x):
    return 1/(1+np.exp(-(w*x+b)))

def error(w,b):
    err = 0
    i=0
    for x,y in zip(X,Y):
        fx = sigmoid(w,b,x)
        err = err+0.5*(fx-y)**2
    return err

def grad_w(w,b,x,y):
    fx = sigmoid(w,b,x)
    return (fx-y)*fx*(1-fx)*x

def grad_b(w,b,x,y):
    fx = sigmoid(w,b,x)
    return (fx-y)*fx*(1-fx)

In [56]:
init_w,init_b=1,1
max_epochs = 100

In [58]:
def momentum_gd():
    w,b,eta = init_w,init_b,1.0
    prev_v_w,prev_v_b,gamma = 0,0,0.9
    for i in range(max_epochs):
        dw,db=0,0
        for x,y in zip(X,Y):
            dw = dw + grad_w(w,b,x,y)
            db = db + grad_b(w,b,x,y)
        
        v_w = gamma*prev_v_w + eta*dw
        v_b = gamma*prev_v_b + eta*db
        w = w - v_w
        b = b - v_b
        prev_v_w = v_w
        prev_v_b = v_b
        err= error(w,b)
    print("momentum_grad:")
    print(err)

In [59]:
def NAG():
    w,b,eta = init_w,init_b,1.0
    prev_v_w,prev_v_b,gamma = 0,0,0.9
    for i in range(max_epochs):
        dw,db=0,0
        #partial updates
        v_w = gamma*prev_v_w
        v_b = gamma*prev_v_b
        for x,y in zip(X,Y):
            dw = dw + grad_w(w-v_w,b-v_b,x,y)
            db = db + grad_b(w-v_w,b-v_b,x,y)
        
        v_w = gamma*prev_v_w + eta*dw
        v_b = gamma*prev_v_b + eta*db
        w = w - v_w
        b = b - v_b
        prev_v_w = v_w
        prev_v_b = v_b
        err= error(w,b)
    print("NAG:")
    print(err)

In [73]:
def mini_batch_gd():
    w,b,eta = -2,-2,1.0
    mini_batch_size,num_points_seen = 2,0
    for i in range(max_epochs):
        dw,db,num_points = 0,0,0
        for x,y in zip(X,Y):
            dw = dw + grad_w(w,b,x,y)
            db = db + grad_b(w,b,x,y)
            num_points_seen += 1
            
            if num_points_seen % mini_batch_size == 0:
                w = w - eta*dw
                b = b - eta*db
                dw,db= 0 , 0
                
        err= error(w,b)
    print("Mini batch:")
    print(err)

In [62]:
def stochastic_gd():
    w,b,eta = -2,-2,1.0
    for i in range(max_epochs):
        dw,db = 0,0
        for x,y in zip(X,Y):
            dw = dw + grad_w(w,b,x,y)
            db = db + grad_b(w,b,x,y)
            w = w - eta*dw
            b = b - eta*db
        err= error(w,b)
    print("stochastic grad:")
    print(err)
        

In [63]:
def accl_momentum_gd():
    w,b,eta = init_w,init_b,1.0
    prev_v_w,prev_v_b,gamma = 0,0,0.9
    for i in range(max_epochs):
        dw,db=0,0
        for x,y in zip(X,Y):
            dw = dw + grad_w(w,b,x,y)
            db = db + grad_b(w,b,x,y)
        
            v_w = gamma*prev_v_w + eta*dw
            v_b = gamma*prev_v_b + eta*db
            w = w - v_w
            b = b - v_b
            prev_v_w = v_w
            prev_v_b = v_b
            err= error(w,b)
    print("momentum accl:")
    print(err)

In [64]:
def accl_NAG():
    w,b,eta = init_w,init_b,1.0
    prev_v_w,prev_v_b,gamma = 0,0,0.9
    for i in range(max_epochs):
        dw,db=0,0
        #partial updates
        v_w = gamma*prev_v_w
        v_b = gamma*prev_v_b
        for x,y in zip(X,Y):
            dw = dw + grad_w(w-v_w,b-v_b,x,y)
            db = db + grad_b(w-v_w,b-v_b,x,y)
        
            v_w = gamma*prev_v_w + eta*dw
            v_b = gamma*prev_v_b + eta*db
            w = w - v_w
            b = b - v_b
            prev_v_w = v_w
            prev_v_b = v_b
            err= error(w,b)
    print("NAG accl:")
    print(err)

In [65]:
def line_serach():
    w,b,etas = init_w ,init_b,[0.1,0.5,1.0,5.0,10.0]
    for i in range(max_epochs):
        dw,db=0,0
        for x,y in zip(X,Y):
            dw += grad_w(w,b,x,y)
            db += grad_b(w,b,x,y)
        min_error = 10000   #set it to a large value
        best_w,best_b = w,b
        for eta in etas:
            tmp_w = w - eta*dw
            tmp_b = b - eta*db
            if error(tmp_w,tmp_b) < min_error:
                best_w = tmp_w
                best_b = tmp_b
                min_error = error(tmp_w,tmp_b)
        w,b = best_w,best_b
        err= error(w,b)
    print("line search:")
    print(err)

In [75]:
def adagrad():
    w,b,eta = init_w,init_b,0.1
    v_w,v_b,eps = 0,0,1e-8
    for i in range(max_epochs):
        dw,db=0,0
        for x,y in zip(X,Y):
            dw += grad_w(w,b,x,y)
            db += grad_b(w,b,x,y)
            
        v_w = v_w + dw**2
        v_b = v_b +dw**2
        
        w = w - (eta/np.sqrt(v_w + eps))*dw
        b = b - (eta/np.sqrt(v_b + eps))*db
        err= error(w,b)
    print("adagrad:")
    print(err)

In [82]:
def rmsprop():
    w,b,eta = init_w,init_b,0.1
    v_w,v_b,eps,beta1= 0,0,1e-8,0.9
    for i in range(max_epochs):
        dw,db=0,0
        for x,y in zip(X,Y):
            dw += grad_w(w,b,x,y)
            db += grad_b(w,b,x,y)
        
        v_w = beta1 * v_w + (1 - beta1)*dw**2
        v_b = beta1 * v_b + (1 - beta1)*db**2
        
        w = w - (eta/np.sqrt(v_w+eps))*dw
        b = b - (eta/np.sqrt(v_b+eps))*db
        err= error(w,b)
    print("rmsprop:")
    print(err)

In [83]:
def adam_optimizer():
    w_b_dw_db = [(1,1,0,0)]
    w_history,b_history,error_history = [],[],[]
    
    w,b,eta,mini_batch_size, num_points_seen = 1,1,0.1,10,0
    
    m_w,m_b,v_w,vb,eps,beta1,beta2 = 0,0,0,0,1e-8,0.9,0.99
    for i in range(100):
        dw,db=0,0
        for x,y in zip(X,Y):
            dw += grad_w(w,b,x,y)
            db += grad_b(w,b,x,y)
            
        m_w = beta1*m_w + (1-beta1)*dw
        m_b = beta1*m_b + (1-beta1)*db
        
        v_w = beta2*v_w + (1-beta2)*(dw**2)
        vb = beta2*vb + (1-beta2)*(db**2)
        
        m_w = m_w/(1 - math.pow(beta1,i+1))
        m_b = m_b/(1 - math.pow(beta1,i+1))
        
        v_w = v_w/(1 - math.pow(beta2,i+1))
        vb = vb/(1 - math.pow(beta2,i+1))
        
        w = w - (eta/np.sqrt(v_w + eps)) * m_w
        b = b - (eta/np.sqrt(vb + eps)) * m_b
        err= error(w,b)
    print("adam:")
    print(err)
        

In [84]:
momentum_gd()
NAG()
stochastic_gd()
mini_batch_gd()
accl_momentum_gd()
accl_NAG()
line_serach
adagrad()
rmsprop()
adam_optimizer()


momentum_grad:
1.0913943873622581e-06
NAG:
5.608446067034885e-09
stochastic grad:
0.038356161578280236
Mini batch:
2.3654398384865547e-05
momentum accl:
0.3052414948646977
NAG accl:
0.30455821299329605
adagrad:
0.001016469025396052
rmsprop:
0.0004439252066547889
adam:
0.015886036334059547
